In [6]:
import sjldpm
from sjldpm.apps.cond.try_cond_1 import *
import importlib
importlib.reload(sjldpm.apps.cond.try_cond_1)

import matplotlib.pyplot as plt

import matplotlib.image as mpimg

2024-10-18 12:53:39.294732: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12
Using TensorFlow backend.


In [7]:
from sjldpm.apps.reference.ddpm2_h import imread,list_pictures

In [8]:
def show_image(path):
    img = mpimg.imread(path)
    imgplot = plt.imshow(img)
    plt.show()

def show_image2(path1,path2):
    img1 = mpimg.imread(path1)
    img2 = mpimg.imread(path2)
    fig,axes= plt.subplots(1,2)
    axes[0].imshow(img1)
    axes[1].imshow(img2)
    plt.show()

def to_displayable_img(img):
    return cv2.cvtColor( (img/2+0.5).astype(np.float32), cv2.COLOR_BGR2RGB)


In [9]:
gpuid = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpuid}"
log_dir = use_log_dir(sjldpm,cfn,"loadPretrained",10)
# ------------------

config  = get_config()
print(" Create model ...")
model = call_by_inspect( get_model, config)
print(" ... Load weights ...")
path_ckpt = Path("/home/yumeng/workspace/DDPM-SuJL/Results-old/ddpm2__v2/version_1/weights/model.weights").as_posix()
model.load_weights(path_ckpt)

 Create model ...



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       128000      input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 128 3456        input_1[0][0]                    
____________________________________________________________________________________________

OSError: Unable to open file (unable to open file: name = '/home/yumeng/workspace/DDPM-SuJL/Results-old/ddpm2__v2/version_1/weights/model.weights', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
def sample_ddim_partial(model, path=None, B=4, z_samples=None, stride=1, eta=1,stop_last_n =1,*,img_size,bar_alpha):
    bar_alpha_ = bar_alpha[::stride]
    bar_alpha_pre_ = np.pad(bar_alpha_[:-1], [1, 0], constant_values=1)
    bar_beta_ = np.sqrt(1 - bar_alpha_**2)
    bar_beta_pre_ = np.sqrt(1 - bar_alpha_pre_**2)
    alpha_ = bar_alpha_ / bar_alpha_pre_
    sigma_ = bar_beta_pre_ / bar_beta_ * np.sqrt(1 - alpha_**2) * eta
    epsilon_ = bar_beta_ - alpha_ * np.sqrt(bar_beta_pre_**2 - sigma_**2)
    T_ = len(bar_alpha_)
    # 采样过程
    if z_samples is None:
        z_samples = np.random.randn(B, img_size, img_size, 3) # Noise
    else:
        z_samples = z_samples.copy()
    for t in tqdm(range(T_ - stop_last_n), ncols=0):
        t = T_ - t - 1
        bt = np.array([[t * stride]] * z_samples.shape[0])
        # z_samples = cond_fn(z_samples,bt,bar_beta_)
        z_samples -= epsilon_[t] * model.predict([z_samples, bt])
        z_samples /= alpha_[t]
        z_samples += np.random.randn(*z_samples.shape) * sigma_[t]
        
    
    h,w = z_samples.shape[1],z_samples.shape[2]
    upz_samples = np.zeros(4*B,img_size,img_size,3)
    for i in range(4):
        upz_samples[i*B:i*B+B,:,:,:] = z_samples
    
    
    # Do the rest of steps
    for t in tqdm(range(T_ - stop_last_n,T_), ncols=0):
        t = T_ - t  - 1
        bt = np.array([[t * stride]] * upz_samples.shape[0])
        # z_samples = cond_fn(z_samples,bt,bar_beta_)
        upz_samples -= epsilon_[t] * model.predict([upz_samples, bt])
        upz_samples /= alpha_[t]
        upz_samples += np.random.randn(*upz_samples.shape) * sigma_[t]
    
    fsz_samples = np.zeros(B,h*4,w*4,3)
    for i in range(4):
        for j in range(B):
            fsz_samples[j,i:h*4:4,i:w*4:4,:] = upz_samples[i*B+j,:,:,:]
    
    
    x_samples = np.clip(fsz_samples, -1, 1)
    fsimg_size= img_size *2
    if path is None:
        return x_samples
    figure = np.zeros((fsimg_size * B, fsimg_size * B, 3))
    for i in range(B):
        for j in range(B):
            digit = x_samples[i * B+ j]
            figure[i * fsimg_size:(i + 1) * fsimg_size,
                   j * fsimg_size:(j + 1) * fsimg_size] = digit
    imwrite(path, figure)
    return figure
    

In [ ]:
sample_ddim_partial(model, path=None, B=4, z_samples=None, stride=1, eta=1,stop_last_n =1,*,img_size,bar_alpha)

In [5]:
import numpy as np
a = np.array(list(range(24)))
b = np.array(list(range(24)))

print(a[0:24:2])

b[0:24:2] =- a[0:24:2]
print(b)

[ 0  2  4  6  8 10 12 14 16 18 20 22]
[  0   1  -2   3  -4   5  -6   7  -8   9 -10  11 -12  13 -14  15 -16  17
 -18  19 -20  21 -22  23]
